<a href="https://colab.research.google.com/github/parmarsuraj99/COMP8730_research_project/blob/main/notebooks/COMP8730_proposed_solution_author_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.18.0
!pip install datasets==2.0.0
!pip install apache_beam==2.37.0
!pip install sentencepiece==0.1.96
!pip install wandb

     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 62.9 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 895 kB 64.6 MB/s 
     |████████████████████████████████| 77 kB 5.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 136 kB 54.5 MB/s 
     |████████████████████████████████| 1.1 MB 65.2 MB/s 
     |████████████████████████████████| 212 kB 85.0 MB/s 
     |████████████████████████████████| 127 kB 74.6 MB/s 
     |████████████████████████████████| 271 kB 9.3 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 144 kB 76.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3

     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 1.8 MB 4.3 MB/s 
     |████████████████████████████████| 181 kB 80.1 MB/s 
     |████████████████████████████████| 144 kB 71.2 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=d15105aeb38ebea97038402fc18c6e944b86ec3d7144534664a42d3932522091
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [2]:
!git clone https://github.com/parmarsuraj99/COMP8730_research_project

Cloning into 'COMP8730_research_project'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 35 (delta 11), reused 18 (delta 4), pack-reused 0
Unpacking objects: 100% (35/35), done.


In [3]:
%cd /content/COMP8730_research_project

/content/COMP8730_research_project


## Author prediction training

In [4]:
import random
random.seed(0)

In [ ]:
import gdown
gdown.download_folder("https://drive.google.com/drive/folders/1vdloyc7skwlIAN5bEG7JdI4Pyu6JaXBU")

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

def prepare_csv(file_dir="inltk_sanskrit_shlokas_dataset"):

    df = pd.read_csv(f"{file_dir}/train.csv")
    df_test = pd.read_csv(f"{file_dir}/valid.csv")

    train_ = df.copy()
    test_ = df_test.copy()

    enc = LabelEncoder()
    train_["Class"] = enc.fit_transform(df["Class"])
    test_["Class"] = enc.transform(df_test["Class"])

    train_.rename(columns={"Class": "label", "Sloka":"text"}, inplace=True)
    test_.rename(columns={"Class": "label", "Sloka":"text"}, inplace=True)

    train_["text"] = train_["text"].str.replace(' +', ' ')
    test_["text"] = test_["text"].str.replace(' +', ' ')

    train_.to_csv("train_processed.csv", index=False)
    test_.to_csv("test_processed.csv", index=False)

prepare_csv()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: The default value of regex will change from True to False in a future version.


In [7]:
from datasets import load_dataset

import glob, os, gc
import numpy as np
from IPython.display import clear_output

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

from transformers import TrainingArguments, Trainer
import transformers
import numpy as np
import torch
from datasets import load_metric
import json


os.environ["WANDB_DISABLED"] = "true"
freeze_all=False

dataset = load_dataset('csv', data_files={"train": 'train_processed.csv', "test":"test_processed.csv"})

Using custom data configuration default-3f2ec5d555eac1dc


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3f2ec5d555eac1dc/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
freeze_all = False

In [12]:
for postfix_ in ["True", "False"]:
    sorted_files = sorted(
        glob.glob(
            f"/content/COMP8730_research_project/COMP8730_NLPU/*/results_scratch_{postfix_}/checkpoint*"
        ),
        key=lambda x: int(x.split("-")[1]),
    )

    for i in range(len(sorted_files)):

        sorted_files_index = i
        config_name = (
            sorted_files[sorted_files_index].split("/")[-2]
            + "_"
            + sorted_files[sorted_files_index].split("/")[-1]
        ).replace("-", "_")

        gc.collect()

        tokenizer = AutoTokenizer.from_pretrained(sorted_files[sorted_files_index])
        model = AutoModelForSequenceClassification.from_pretrained(
            sorted_files[sorted_files_index], num_labels=3
        )
        gc.collect()

        clear_output()
        gc.collect()

        def tokenize_function(examples):
            return tokenizer(
                examples["text"].replace("\n", "").replace("\t", ""),
                padding="max_length",
                truncation=True,
                max_length=128,
            )

        tokenized_datasets = dataset.map(tokenize_function)

        if not freeze_all:
            for name, param in list(model.albert.named_parameters())[:-5]:
                param.requires_grad = False
        else:
            model.albert.requires_grad_(False)

        metric = load_metric("f1")

        def compute_metrics(eval_pred):
            metric1 = load_metric("precision")
            metric2 = load_metric("recall")
            metric_f1 = load_metric("f1")
            metric_acc = load_metric("accuracy")
            
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)

            precision = metric1.compute(predictions=predictions, references=labels, average='weighted')["precision"]
            recall = metric2.compute(predictions=predictions, references=labels, average='weighted')["recall"]
            f1 = metric_f1.compute(predictions=predictions, references=labels, average='weighted')["f1"]
            accuracy = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]

            return {"precision": precision, "recall": recall, "f1":f1, "accuracy":accuracy}

        op_dir = (
            sorted_files[sorted_files_index].split("/")[-2]
            + "_"
            + sorted_files[sorted_files_index].split("/")[-1]
        )

        training_args = TrainingArguments(
            output_dir=op_dir,
            num_train_epochs=10,
            do_train=True,
            do_eval=True,
            logging_strategy="epoch",
            optim="adamw_torch",
            save_strategy="epoch",
            save_total_limit=1,
            learning_rate=1e-6,
            evaluation_strategy="epoch",
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            data_seed=0,
            load_best_model_at_end=True,
        )

        gc.collect()

        gc.collect()
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"],
            compute_metrics=compute_metrics,
        )
        gc.collect()

        results_dict = dict()

        train_stats = trainer.train()
        results_dict["train_stats"] = train_stats
        gc.collect()

        eval_scores = trainer.evaluate()
        results_dict["eval_scores"] = eval_scores
        gc.collect()

        train_scores = trainer.evaluate(tokenized_datasets["train"])
        results_dict["train_scores"] = train_scores
        gc.collect()

        eval_preds = trainer.predict(tokenized_datasets["test"])

        eval_tgt = np.array(tokenized_datasets["test"]["label"])

        with open(f"{config_name}.json", "w") as fp:
            json.dump(results_dict, fp)


Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-3f2ec5d555eac1dc/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-e3bce887c1805a4c.arrow


  0%|          | 0/96 [00:00<?, ?ex/s]

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
The following columns in the training set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 383
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3830


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.100900,1.043223,0.305678,0.458333,0.343478,0.458333
2,0.963300,0.839335,0.513844,0.687500,0.581076,0.687500


The following columns in the evaluation set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 96
  Batch size = 1


Saving model checkpoint to results_scratch_True_checkpoint-1455/checkpoint-383
Configuration saved in results_scratch_True_checkpoint-1455/checkpoint-383/config.json
Model weights saved in results_scratch_True_checkpoint-1455/checkpoint-383/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 96
  Batch size = 1
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to results_scratch_True_checkpoint-1455/checkpoint-766
Configuration saved in results_sc

KeyboardInterrupt: ignored

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile("results.zip", 'w') as myZip:
    for name in glob.glob("*.json" ):
        myZip.write(name, os.path.basename(name), zipfile.ZIP_DEFLATED)

In [ ]:
results_dict = {}
for file_ in glob.glob("*.json"):
    with open(file_, "r") as fp:
        res = json.load(fp)
    tmp = {}
    tmp["eval_f1"] = res["eval_scores"]["eval_f1"]
    results_dict[file_.split(".")[0]] = tmp

In [ ]:
results_dict

In [ ]:
res_df = pd.DataFrame.from_dict(results_dict).T

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(res_df.loc[[c for c in res_df.index.to_list() if "False" in c]].mean())
res_df.loc[[c for c in res_df.index.to_list() if "False" in c]].plot.barh(figsize=(10,6))
plt.show()

In [ ]:
print(res_df.loc[[c for c in res_df.index.to_list() if "True" in c]].mean())
res_df.loc[[c for c in res_df.index.to_list() if "True" in c]].plot.barh(figsize=(10,6))
plt.show()

## Uploading to HF Hub

optional if you'd like to share your models

In [ ]:
!pip install huggingface_hub
!sudo apt-get install git-lfs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
random.seed(0)
import glob

from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
for postfix_ in ["True", "False"]:
    sorted_files = sorted(
        glob.glob(
            f"/content/COMP8730_research_project/results_scratch_{postfix_}/*/checkpoint*"
        ),
        key=lambda x: int(x.split("-")[1]),
    )
    print(sorted_files[-1])

    if postfix_=="False":
        model_hub_name = "sanbert-from-indicbert"
    else:
        model_hub_name = "sanbert-from-scratch"

    tokenizer = AutoTokenizer.from_pretrained(sorted_files[-1])
    model = AutoModelForMaskedLM.from_pretrained(
            sorted_files[-1]
    )

    model.push_to_hub(model_hub_name, use_temp_dir=True)
    tokenizer.push_to_hub(model_hub_name, use_temp_dir=True)